In [1]:
!pip install torch numpy scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

In [27]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays
import random

# --- Static Params ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
EPOCHS = 20
PATIENCE = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_COMBINATIONS = 20
TRAIN_FRACTION = 0.01
EMBED_DIM = 8

# --- Load data ---
df = pd.read_csv('bicikelj_train.csv')
meta = pd.read_csv('bicikelj_metadata.csv')
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Station normalization ---
station_means = df[station_cols].mean()
station_stds = df[station_cols].std().replace(0, 1)
df_norm = df.copy()
df_norm[station_cols] = (df[station_cols] - station_means) / station_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])

        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)
        time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                               month_sin, month_cos, is_weekend, is_holiday], axis=1)

        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)

        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)

            for i in range(history_len, N - pred_horizon + 1):
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]  # crop end
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x, station_id):
        # x: [B, T, D] → [B, D, T]
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]  # [B, C]
        emb = self.embedding(station_id)  # [B, E]
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Training ---
def train_tcn(model, train_loader, val_loader, lr, weight_decay):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    best_loss = float('inf')
    best_state = None
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        for xb, yb, sid in train_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb, sid), yb)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb, sid in val_loader:
                xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
                val_loss += criterion(model(xb, sid), yb).item()
        val_loss /= len(val_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    return model, best_loss

# --- Grid search ---
param_grid = {
    # 'hidden_dim': [32, 64, 128],
    # 'dropout': [0.0, 0.1, 0.2],
    # 'lr': [1e-3, 5e-4, 2e-4],
    # 'weight_decay': [0.0, 1e-5, 1e-4]
    'hidden_dim': [64],
    'dropout': [0.2],
    'lr': [0.0005],
    'weight_decay': [0.0001]
}
param_combos = list(itertools.product(*param_grid.values()))
random.shuffle(param_combos)
param_combos = param_combos[:MAX_COMBINATIONS]

# --- Dataset ---
dataset = SharedTCNDataset(df_norm, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON)
N = len(dataset)
reduced_N = int(N * TRAIN_FRACTION)
indices = list(range(N))
random.shuffle(indices)

train_size = int(reduced_N * 0.7)
val_size = int(reduced_N * 0.15)
holdout_size = reduced_N - train_size - val_size

train_set = Subset(dataset, indices[:train_size])
val_set = Subset(dataset, indices[train_size:train_size + val_size])
holdout_set = Subset(dataset, indices[train_size + val_size:train_size + val_size + holdout_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64)
holdout_loader = DataLoader(holdout_set, batch_size=64)

# --- Run ---
input_dim = 1 + K_NEIGHBORS + 8  # station + neighbors + time features
output_dim = PRED_HORIZON
num_stations = len(station_cols)

n_layers = 4

results = []
print(f"⏳ Running grid search over {len(param_combos)} combinations...")
for i, (hdim, dr, lr, wd) in enumerate(param_combos):
    print(f"\n🔍 Combo {i+1}: hidden_dim={hdim}, dropout={dr}, lr={lr}, weight_decay={wd}")
    model = TCN(input_size=input_dim,
                output_size=output_dim,
                #num_channels=[hdim] * 3,
                num_channels=[hdim] * n_layers,

                kernel_size=3,
                dropout=dr,
                num_stations=num_stations,
                embed_dim=EMBED_DIM)
    model, val_loss = train_tcn(model, train_loader, val_loader, lr, wd)

    model.eval()
    holdout_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for xb, yb, sid in holdout_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            holdout_loss += criterion(model(xb, sid), yb).item()
    holdout_loss /= len(holdout_loader)

    print(f"✅ Val Loss: {val_loss:.4f}, Holdout Loss: {holdout_loss:.4f}")
    results.append({
        "hidden_dim": hdim,
        "dropout": dr,
        "lr": lr,
        "weight_decay": wd,
        "val_loss": val_loss,
        "holdout_loss": holdout_loss
    })

# --- Save ---
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="holdout_loss")
results_df.to_csv("grid_search_tcn_results.csv", index=False)
print("\n📊 Top 5 Results:")
print(results_df.head())


⏳ Running grid search over 1 combinations...

🔍 Combo 1: hidden_dim=64, dropout=0.2, lr=0.0005, weight_decay=0.0001
✅ Val Loss: 0.3289, Holdout Loss: 0.3381

📊 Top 5 Results:
   hidden_dim  dropout      lr  weight_decay  val_loss  holdout_loss
0          64      0.2  0.0005        0.0001  0.328911      0.338051


# Complete

In [12]:
# --- TCN Bicikelj final training + test prediction with speedups ---

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays
import random

# --- Hyperparameters ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMBED_DIM = 8
HIDDEN_DIM = 64
N_LAYERS = 3
LR = 0.0005
WEIGHT_DECAY = 0.0001
DROPOUT = 0.2
EPOCHS = 50
PATIENCE = 8
BATCH_SIZE = 128  # increased safely

# --- Load data ---
df = pd.read_csv("bicikelj_train.csv")
meta = pd.read_csv("bicikelj_metadata.csv")
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# Station normalization
station_means = df[station_cols].mean()
station_stds = df[station_cols].std().replace(0, 1)
df_norm = df.copy()
df_norm[station_cols] = (df[station_cols] - station_means) / station_stds

# Neighbors
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])

        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.sin(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)
        time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                               month_sin, month_cos, is_weekend, is_holiday], axis=1)

        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)

        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)

            for i in range(history_len, N - pred_horizon + 1):
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN Block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Create Dataset and split ---
dataset = SharedTCNDataset(df_norm, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON)

N = len(dataset)
indices = list(range(N))
random.shuffle(indices)

val_size = int(0.1 * N)  # 10% for val
train_size = N - val_size

train_indices = indices[:train_size]
val_indices = indices[train_size:]

train_set = Subset(dataset, train_indices)
val_set = Subset(dataset, val_indices)

# --- DataLoaders with speedups ---
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)

# --- Model ---
model = TCN(input_size=1 + K_NEIGHBORS + 8,
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)

# --- Optimizer and Loss ---
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

# --- Training loop ---
best_loss = float('inf')
best_state = None
patience_counter = 0

for epoch in range(EPOCHS):
    # --- Train ---
    model.train()
    running_loss = 0.0
    for xb, yb, sid in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(xb, sid), yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb, sid in val_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            val_loss += criterion(model(xb, sid), yb).item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}: Val Loss = {avg_val_loss:.4f}")

    # --- Early stopping on val loss ---
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

# --- Save best model ---
model.load_state_dict(best_state)
torch.save(model.state_dict(), "tcn_model_final.pt")
print("✅ Saved model to 'tcn_model_final.pt'")

# --- Predict on test set ---
# (keep your current test loop as is → no changes needed there)

# --- Predict on bicikelj_test.csv ---
test_df = pd.read_csv("bicikelj_test.csv")
test_feats = test_df[station_cols].values.astype(np.float32)
timestamps = pd.to_datetime(test_df["timestamp"])

# Time features
hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
slo_holidays = holidays.Slovenia()
is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                       month_sin, month_cos, is_weekend, is_holiday], axis=1)

name_to_idx = {name: i for i, name in enumerate(station_cols)}

# Load model for inference
model.eval()

pred_matrix = np.full_like(test_feats, np.nan)

with torch.no_grad():
    for i in range(HISTORY_LEN, len(test_df) - PRED_HORIZON + 1):
        if np.isnan(test_feats[i:i + PRED_HORIZON]).all(axis=0).all():
            for station in station_cols:
                s_idx = name_to_idx[station]
                nn_idx = [name_to_idx[nn] for nn in neighbors[station]]

                seq = []
                for t in range(i - HISTORY_LEN, i):
                    row = [test_feats[t, s_idx]]
                    row += [test_feats[t, j] for j in nn_idx]
                    row += list(time_feats[t])
                    seq.append(row)
                seq = torch.tensor([seq], dtype=torch.float32).to(DEVICE)

                pred_norm = model(seq, torch.tensor([s_idx], dtype=torch.long, device=DEVICE)).cpu().numpy().flatten()
                pred = pred_norm * station_stds[station] + station_means[station]

                for j in range(PRED_HORIZON):
                    pred_matrix[i + j, s_idx] = pred[j]

# Save predictions
pred_df = pd.DataFrame(pred_matrix, columns=station_cols)
pred_df.insert(0, "timestamp", test_df["timestamp"])
rows_to_output = test_df[station_cols].isna().all(axis=1)
pred_df_filtered = pred_df[rows_to_output].copy()
pred_df_filtered.to_csv("bicikelj_test_predictions_tcn.csv", index=False)
print("✅ Saved predictions to 'bicikelj_test_predictions_tcn.csv'")


Epoch 1/50: 100%|██████████| 12056/12056 [01:23<00:00, 144.69it/s]

Epoch 1: Train Loss = 0.3354


Epoch 1: Val Loss = 0.3062


Epoch 2/50: 100%|██████████| 12056/12056 [01:22<00:00, 146.26it/s]

Epoch 2: Train Loss = 0.3052


...

Epoch 45: Train Loss = 0.2880


Epoch 45: Val Loss = 0.2842


Epoch 46/50: 100%|██████████| 12056/12056 [01:23<00:00, 144.57it/s]

Epoch 46: Train Loss = 0.2879


Epoch 46: Val Loss = 0.2850
Early stopping!
✅ Saved model to 'tcn_model_final.pt'
✅ Saved predictions to 'bicikelj_test_predictions_tcn.csv'


In [13]:
# --- Predict only the unknown rows in bicikelj_test.csv using final TCN model ---

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.metrics.pairwise import haversine_distances
import holidays

# --- Constants ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMBED_DIM = 8
HIDDEN_DIM = 64
N_LAYERS = 3
DROPOUT = 0.2
BATCH_SIZE = 128

# --- Load metadata ---
meta = pd.read_csv("bicikelj_metadata.csv")
station_cols = pd.read_csv("bicikelj_test.csv").columns[1:]
station_names = meta['name'].tolist()

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Load training stats for normalization ---
df_train = pd.read_csv("bicikelj_train.csv")
station_means = df_train[station_cols].mean()
station_stds = df_train[station_cols].std().replace(0, 1)

# --- TCN Model definition ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Load model ---
model = TCN(input_size=1 + K_NEIGHBORS + 8,
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)

model.load_state_dict(torch.load("tcn_model_final.pt"))
model.eval()

# --- Load test set ---
test_df = pd.read_csv("bicikelj_test.csv")
test_feats = test_df[station_cols].values.astype(np.float32)
timestamps = pd.to_datetime(test_df["timestamp"])

# --- Time features ---
hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
slo_holidays = holidays.Slovenia()
is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                       month_sin, month_cos, is_weekend, is_holiday], axis=1)

# --- Normalize test_feats using training stats ---
test_feats_norm = (test_feats - station_means.values) / station_stds.values

# --- Predict ---
name_to_idx = {name: i for i, name in enumerate(station_cols)}

pred_matrix = np.full_like(test_feats, np.nan)

with torch.no_grad():
    for i in range(HISTORY_LEN, len(test_df) - PRED_HORIZON + 1):
        if np.isnan(test_feats[i:i + PRED_HORIZON]).all(axis=0).all():
            for station in station_cols:
                s_idx = name_to_idx[station]
                nn_idx = [name_to_idx[nn] for nn in neighbors[station]]

                seq = []
                for t in range(i - HISTORY_LEN, i):
                    row = [test_feats_norm[t, s_idx]]
                    row += [test_feats_norm[t, j] for j in nn_idx]
                    row += list(time_feats[t])
                    seq.append(row)

                seq = torch.tensor([seq], dtype=torch.float32).to(DEVICE)
                sid_tensor = torch.tensor([s_idx], dtype=torch.long, device=DEVICE)

                pred_norm = model(seq, sid_tensor).cpu().numpy().flatten()
                pred = pred_norm * station_stds[station] + station_means[station]

                for j in range(PRED_HORIZON):
                    pred_matrix[i + j, s_idx] = pred[j]

# --- Save predictions ---
pred_df = pd.DataFrame(pred_matrix, columns=station_cols)
pred_df.insert(0, "timestamp", test_df["timestamp"])

rows_to_output = test_df[station_cols].isna().all(axis=1)
pred_df_filtered = pred_df[rows_to_output].copy()

pred_df_filtered.to_csv("bicikelj_test_predictions_tcn.csv", index=False)
print("✅ Saved predictions to 'bicikelj_test_predictions_tcn.csv'")


✅ Saved predictions to 'bicikelj_test_predictions_tcn.csv'


# Multiheaded output

In [17]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays
import random

# --- Static Params ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
EPOCHS = 20
PATIENCE = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_COMBINATIONS = 20
TRAIN_FRACTION = 0.01
EMBED_DIM = 8

# --- Load data ---
df = pd.read_csv('bicikelj_train.csv')
meta = pd.read_csv('bicikelj_metadata.csv')
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Station normalization ---
station_means = df[station_cols].mean()
station_stds = df[station_cols].std().replace(0, 1)
df_norm = df.copy()
df_norm[station_cols] = (df[station_cols] - station_means) / station_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])

        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)
        time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                               month_sin, month_cos, is_weekend, is_holiday], axis=1)

        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)

        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)

            for i in range(history_len, N - pred_horizon + 1):
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]  # crop end
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)

        # Multihead
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(num_channels[-1] + embed_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(output_size)
        ])

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)

        out = [head(combined) for head in self.heads]  # list of [B,1]
        out = torch.cat(out, dim=1)  # [B, PRED_HORIZON]
        return out


# --- Training ---
def train_tcn(model, train_loader, val_loader, lr, weight_decay):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    best_loss = float('inf')
    best_state = None
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        for xb, yb, sid in train_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb, sid), yb)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb, sid in val_loader:
                xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
                val_loss += criterion(model(xb, sid), yb).item()
        val_loss /= len(val_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    return model, best_loss

# --- Grid search ---
param_grid = {
    'hidden_dim': [32, 64, 128],
    'dropout': [0.0, 0.1, 0.2],
    'lr': [1e-3, 5e-4, 2e-4],
    'weight_decay': [0.0, 1e-5, 1e-4]
    # 'hidden_dim': [64],
    # 'dropout': [0.2],
    # 'lr': [0.0005],
    # 'weight_decay': [0.0001]
}
param_combos = list(itertools.product(*param_grid.values()))
random.shuffle(param_combos)
param_combos = param_combos[:MAX_COMBINATIONS]

# --- Dataset ---
dataset = SharedTCNDataset(df_norm, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON)
N = len(dataset)
reduced_N = int(N * TRAIN_FRACTION)
indices = list(range(N))
random.shuffle(indices)

train_size = int(reduced_N * 0.7)
val_size = int(reduced_N * 0.15)
holdout_size = reduced_N - train_size - val_size

train_set = Subset(dataset, indices[:train_size])
val_set = Subset(dataset, indices[train_size:train_size + val_size])
holdout_set = Subset(dataset, indices[train_size + val_size:train_size + val_size + holdout_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64)
holdout_loader = DataLoader(holdout_set, batch_size=64)

# --- Run ---
input_dim = 1 + K_NEIGHBORS + 8  # station + neighbors + time features
output_dim = PRED_HORIZON
num_stations = len(station_cols)

n_layers = 4

results = []
print(f"⏳ Running grid search over {len(param_combos)} combinations...")
for i, (hdim, dr, lr, wd) in enumerate(param_combos):
    print(f"\n🔍 Combo {i+1}: hidden_dim={hdim}, dropout={dr}, lr={lr}, weight_decay={wd}")
    model = TCN(input_size=input_dim,
                output_size=output_dim,
                #num_channels=[hdim] * 3,
                num_channels=[hdim] * n_layers,

                kernel_size=3,
                dropout=dr,
                num_stations=num_stations,
                embed_dim=EMBED_DIM)
    model, val_loss = train_tcn(model, train_loader, val_loader, lr, wd)

    model.eval()
    holdout_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for xb, yb, sid in holdout_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            holdout_loss += criterion(model(xb, sid), yb).item()
    holdout_loss /= len(holdout_loader)

    print(f"✅ Val Loss: {val_loss:.4f}, Holdout Loss: {holdout_loss:.4f}")
    results.append({
        "hidden_dim": hdim,
        "dropout": dr,
        "lr": lr,
        "weight_decay": wd,
        "val_loss": val_loss,
        "holdout_loss": holdout_loss
    })

# --- Save ---
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="holdout_loss")
results_df.to_csv("grid_search_tcn_results.csv", index=False)
print("\n📊 Top 5 Results:")
print(results_df.head())


⏳ Running grid search over 20 combinations...

🔍 Combo 1: hidden_dim=32, dropout=0.0, lr=0.0005, weight_decay=0.0001
✅ Val Loss: 0.3813, Holdout Loss: 0.4025

🔍 Combo 2: hidden_dim=64, dropout=0.1, lr=0.0002, weight_decay=0.0001
✅ Val Loss: 0.3615, Holdout Loss: 0.3727

🔍 Combo 3: hidden_dim=128, dropout=0.0, lr=0.0002, weight_decay=1e-05
✅ Val Loss: 0.3846, Holdout Loss: 0.4234

🔍 Combo 4: hidden_dim=64, dropout=0.1, lr=0.001, weight_decay=1e-05
✅ Val Loss: 0.3610, Holdout Loss: 0.3699

🔍 Combo 5: hidden_dim=64, dropout=0.2, lr=0.0005, weight_decay=1e-05
✅ Val Loss: 0.3588, Holdout Loss: 0.3673

🔍 Combo 6: hidden_dim=32, dropout=0.1, lr=0.001, weight_decay=0.0
✅ Val Loss: 0.3616, Holdout Loss: 0.3744

🔍 Combo 7: hidden_dim=128, dropout=0.2, lr=0.0005, weight_decay=0.0001
✅ Val Loss: 0.3616, Holdout Loss: 0.3730

🔍 Combo 8: hidden_dim=32, dropout=0.0, lr=0.001, weight_decay=0.0001


KeyboardInterrupt: 

# Weather

In [26]:
import itertools
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays
import random

# --- Static Params ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
EPOCHS = 20
PATIENCE = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_COMBINATIONS = 20
TRAIN_FRACTION = 0.01
EMBED_DIM = 8

# --- Load data ---
df = pd.read_csv('bicikelj_train.csv')
meta = pd.read_csv('bicikelj_metadata.csv')
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Load weather ---
# Load weather
weather_df = pd.read_csv("weather_ljubljana.csv", skiprows=2)
weather_df = weather_df.rename(columns={
    'temperature_2m (°C)': 'temperature_2m',
    'precipitation (mm)': 'precipitation',
    'windspeed_10m (km/h)': 'windspeed_10m',
    'cloudcover (%)': 'cloudcover'
})
weather_df['time'] = pd.to_datetime(weather_df['time'])

# Align timestamp formats
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)

# Merge
df_merged = pd.merge(df, weather_df, left_on='timestamp', right_on='time', how='left')


# Weather features to use
weather_features = ['temperature_2m', 'precipitation', 'windspeed_10m', 'cloudcover']
df_merged[weather_features] = df_merged[weather_features].ffill().bfill()

# Normalize station + weather
station_means = df_merged[station_cols].mean()
station_stds = df_merged[station_cols].std().replace(0, 1)
df_norm = df_merged.copy()
df_norm[station_cols] = (df_merged[station_cols] - station_means) / station_stds

weather_means = df_merged[weather_features].mean()
weather_stds = df_merged[weather_features].std().replace(0, 1)
df_norm[weather_features] = (df_merged[weather_features] - weather_means) / weather_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon, weather_features):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])

        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.cos(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

        weather_array = df[weather_features].values  # [N, W]

        time_feats = np.concatenate([
            np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                      month_sin, month_cos, is_weekend, is_holiday], axis=1),
            weather_array
        ], axis=1)  # [N, 8+W]

        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)

        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)

            for i in range(history_len, N - pred_horizon + 1):
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv1d):
                nn.init.kaiming_normal_(m.weight)

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)

        # Multihead:
        self.heads = nn.ModuleList([
            nn.Sequential(
                nn.Linear(num_channels[-1] + embed_dim, 64),
                nn.ReLU(),
                nn.Linear(64, 1)
            ) for _ in range(output_size)
        ])

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        out = [head(combined) for head in self.heads]
        out = torch.cat(out, dim=1)
        return out

# --- Training ---
def train_tcn(model, train_loader, val_loader, lr, weight_decay):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    best_loss = float('inf')
    best_state = None
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        for xb, yb, sid in train_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(xb, sid), yb)
            loss.backward()
            optimizer.step()

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for xb, yb, sid in val_loader:
                xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
                val_loss += criterion(model(xb, sid), yb).item()
        val_loss /= len(val_loader)

        if val_loss < best_loss:
            best_loss = val_loss
            best_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= PATIENCE:
                break

    model.load_state_dict(best_state)
    return model, best_loss

# --- Grid search ---
param_grid = {
    'hidden_dim': [64],
    'dropout': [0.2],
    'lr': [0.0005],
    'weight_decay': [0.0001]
}
param_combos = list(itertools.product(*param_grid.values()))
random.shuffle(param_combos)
param_combos = param_combos[:MAX_COMBINATIONS]

# --- Dataset ---
dataset = SharedTCNDataset(df_norm, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON, weather_features)
N = len(dataset)
reduced_N = int(N * TRAIN_FRACTION)
indices = list(range(N))
random.shuffle(indices)

train_size = int(reduced_N * 0.7)
val_size = int(reduced_N * 0.15)
holdout_size = reduced_N - train_size - val_size

train_set = Subset(dataset, indices[:train_size])
val_set = Subset(dataset, indices[train_size:train_size + val_size])
holdout_set = Subset(dataset, indices[train_size + val_size:train_size + val_size + holdout_size])

train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = DataLoader(val_set, batch_size=64)
holdout_loader = DataLoader(holdout_set, batch_size=64)

# --- Run ---
input_dim = 1 + K_NEIGHBORS + (8 + len(weather_features))
output_dim = PRED_HORIZON
num_stations = len(station_cols)
n_layers = 4

results = []
print(f"⏳ Running grid search over {len(param_combos)} combinations...")
for i, (hdim, dr, lr, wd) in enumerate(param_combos):
    print(f"\n🔍 Combo {i+1}: hidden_dim={hdim}, dropout={dr}, lr={lr}, weight_decay={wd}")
    model = TCN(input_size=input_dim,
                output_size=output_dim,
                num_channels=[hdim] * n_layers,
                kernel_size=3,
                dropout=dr,
                num_stations=num_stations,
                embed_dim=EMBED_DIM)
    model, val_loss = train_tcn(model, train_loader, val_loader, lr, wd)

    model.eval()
    holdout_loss = 0.0
    criterion = nn.MSELoss()
    with torch.no_grad():
        for xb, yb, sid in holdout_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            holdout_loss += criterion(model(xb, sid), yb).item()
    holdout_loss /= len(holdout_loader)

    print(f"✅ Val Loss: {val_loss:.4f}, Holdout Loss: {holdout_loss:.4f}")
    results.append({
        "hidden_dim": hdim,
        "dropout": dr,
        "lr": lr,
        "weight_decay": wd,
        "val_loss": val_loss,
        "holdout_loss": holdout_loss
    })

# --- Save ---
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="holdout_loss")
results_df.to_csv("grid_search_tcn_results.csv", index=False)
print("\n📊 Top 5 Results:")
print(results_df.head())


⏳ Running grid search over 1 combinations...

🔍 Combo 1: hidden_dim=64, dropout=0.2, lr=0.0005, weight_decay=0.0001
✅ Val Loss: 0.3718, Holdout Loss: 0.3555

📊 Top 5 Results:
   hidden_dim  dropout      lr  weight_decay  val_loss  holdout_loss
0          64      0.2  0.0005        0.0001  0.371804      0.355488


# Combined weather

In [29]:
# --- TCN Bicikelj final training + test prediction with weather ---

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import holidays
import random

# --- Hyperparameters ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
EMBED_DIM = 8
HIDDEN_DIM = 64
N_LAYERS = 4
LR = 0.0005
WEIGHT_DECAY = 0.0001
DROPOUT = 0.2
EPOCHS = 50
PATIENCE = 8
BATCH_SIZE = 128

# --- Load data ---
df = pd.read_csv("bicikelj_train.csv")
meta = pd.read_csv("bicikelj_metadata.csv")
station_cols = df.columns[1:]

# Clean and fill
for col in station_cols:
    df[col] = pd.to_numeric(df[col], errors="coerce")
df[station_cols] = df[station_cols].ffill().bfill()
df = df.dropna(subset=station_cols, how='all').reset_index(drop=True)

# --- Load weather ---
weather_df = pd.read_csv("weather_ljubljana.csv", skiprows=2)
weather_df = weather_df.rename(columns={
    'temperature_2m (°C)': 'temperature_2m',
    'precipitation (mm)': 'precipitation',
    'windspeed_10m (km/h)': 'windspeed_10m',
    'cloudcover (%)': 'cloudcover'
})
weather_df['time'] = pd.to_datetime(weather_df['time'])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.tz_localize(None)
df_merged = pd.merge(df, weather_df, left_on='timestamp', right_on='time', how='left')

weather_features = ['temperature_2m', 'precipitation', 'windspeed_10m', 'cloudcover']
df_merged[weather_features] = df_merged[weather_features].ffill().bfill()

# --- Normalize ---
station_means = df_merged[station_cols].mean()
station_stds = df_merged[station_cols].std().replace(0, 1)
df_norm = df_merged.copy()
df_norm[station_cols] = (df_merged[station_cols] - station_means) / station_stds

weather_means = df_merged[weather_features].mean()
weather_stds = df_merged[weather_features].std().replace(0, 1)
df_norm[weather_features] = (df_merged[weather_features] - weather_means) / weather_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- Dataset ---
class SharedTCNDataset(Dataset):
    def __init__(self, df, station_cols, neighbors, history_len, pred_horizon, weather_features):
        self.samples = []
        self.station_to_idx = {name: i for i, name in enumerate(station_cols)}
        timestamps = pd.to_datetime(df['timestamp'])

        hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
        hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
        dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
        dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
        month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
        month_cos = np.sin(2 * np.pi * timestamps.dt.month / 12)
        is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
        slo_holidays = holidays.Slovenia()
        is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

        weather_array = df[weather_features].values

        time_feats = np.concatenate([
            np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                      month_sin, month_cos, is_weekend, is_holiday], axis=1),
            weather_array
        ], axis=1)

        bikes = df[station_cols].values.astype(np.float32)
        N = len(df)

        for s_name in station_cols:
            s_idx = self.station_to_idx[s_name]
            nn_idx = [self.station_to_idx[nn] for nn in neighbors[s_name]]
            series = bikes[:, [s_idx] + nn_idx]
            full_feats = np.concatenate([series, time_feats], axis=1)

            for i in range(history_len, N - pred_horizon + 1):
                x = full_feats[i - history_len:i]
                y = bikes[i:i + pred_horizon, s_idx]
                self.samples.append((x, y, s_idx))

    def __len__(self): return len(self.samples)
    def __getitem__(self, idx):
        x, y, sid = self.samples[idx]
        return (torch.tensor(x, dtype=torch.float32),
                torch.tensor(y, dtype=torch.float32),
                torch.tensor(sid, dtype=torch.long))

# --- TCN Block ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Training ---
dataset = SharedTCNDataset(df_norm, station_cols, neighbors, HISTORY_LEN, PRED_HORIZON, weather_features)
N = len(dataset)
indices = list(range(N))
random.shuffle(indices)

val_size = int(0.1 * N)
train_size = N - val_size

train_indices = indices[:train_size]
val_indices = indices[train_size:]

train_set = Subset(dataset, train_indices)
val_set = Subset(dataset, val_indices)

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, num_workers=4, pin_memory=True)

model = TCN(input_size=1 + K_NEIGHBORS + (8 + len(weather_features)),
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.MSELoss()

best_loss = float('inf')
best_state = None
patience_counter = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for xb, yb, sid in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):
        xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
        optimizer.zero_grad()
        loss = criterion(model(xb, sid), yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_train_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Train Loss = {avg_train_loss:.4f}")

    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for xb, yb, sid in val_loader:
            xb, yb, sid = xb.to(DEVICE), yb.to(DEVICE), sid.to(DEVICE)
            val_loss += criterion(model(xb, sid), yb).item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Epoch {epoch+1}: Val Loss = {avg_val_loss:.4f}")

    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        best_state = model.state_dict()
        patience_counter = 0
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping!")
            break

model.load_state_dict(best_state)
torch.save(model.state_dict(), "tcn_model_final_weather.pt")
print("✅ Saved model to 'tcn_model_final_weather.pt'")

# PREDICTION
# --- Load model ---
model = TCN(input_size=1 + K_NEIGHBORS + (8 + len(weather_features)),
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)

model.load_state_dict(torch.load("tcn_model_final_weather.pt"))
model.eval()

# --- Load test set ---
test_df = pd.read_csv("bicikelj_test.csv")
test_feats = test_df[station_cols].values.astype(np.float32)
timestamps = pd.to_datetime(test_df["timestamp"])

# --- Load weather for test ---
weather_test_df = pd.read_csv("weather_ljubljana_test.csv", skiprows=2)
weather_test_df = weather_test_df.rename(columns={
    'temperature_2m (°C)': 'temperature_2m',
    'precipitation (mm)': 'precipitation',
    'windspeed_10m (km/h)': 'windspeed_10m',
    'cloudcover (%)': 'cloudcover'
})
weather_test_df['time'] = pd.to_datetime(weather_test_df['time'])
test_df['timestamp'] = pd.to_datetime(test_df['timestamp']).dt.tz_localize(None)
test_df_merged = pd.merge(test_df, weather_test_df, left_on='timestamp', right_on='time', how='left')
test_df_merged[weather_features] = test_df_merged[weather_features].ffill().bfill()

# --- Time features ---
hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
month_cos = np.sin(2 * np.pi * timestamps.dt.month / 12)
is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
slo_holidays = holidays.Slovenia()
is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                       month_sin, month_cos, is_weekend, is_holiday], axis=1)

# --- Normalize test_feats and weather ---
test_feats_norm = (test_feats - station_means.values) / station_stds.values
weather_feats_norm = (test_df_merged[weather_features].values - weather_means.values) / weather_stds.values

# --- Predict ---
name_to_idx = {name: i for i, name in enumerate(station_cols)}
pred_matrix = np.full_like(test_feats, np.nan)

with torch.no_grad():
    for i in range(HISTORY_LEN, len(test_df) - PRED_HORIZON + 1):
        if np.isnan(test_feats[i:i + PRED_HORIZON]).all(axis=0).all():
            for station in station_cols:
                s_idx = name_to_idx[station]
                nn_idx = [name_to_idx[nn] for nn in neighbors[station]]

                seq = []
                for t in range(i - HISTORY_LEN, i):
                    row = [test_feats_norm[t, s_idx]]
                    row += [test_feats_norm[t, j] for j in nn_idx]
                    row += list(time_feats[t])
                    row += list(weather_feats_norm[t])
                    seq.append(row)

                seq = torch.tensor([seq], dtype=torch.float32).to(DEVICE)
                sid_tensor = torch.tensor([s_idx], dtype=torch.long, device=DEVICE)

                pred_norm = model(seq, sid_tensor).cpu().numpy().flatten()
                pred = pred_norm * station_stds[station] + station_means[station]

                for j in range(PRED_HORIZON):
                    pred_matrix[i + j, s_idx] = pred[j]

# --- Save predictions ---
pred_df = pd.DataFrame(pred_matrix, columns=station_cols)
pred_df.insert(0, "timestamp", test_df["timestamp"])

rows_to_output = test_df[station_cols].isna().all(axis=1)
pred_df_filtered = pred_df[rows_to_output].copy()

pred_df_filtered.to_csv("bicikelj_test_predictions_tcn_weather.csv", index=False)
print("✅ Saved predictions to 'bicikelj_test_predictions_tcn_weather.csv'")



Epoch 1/50: 100%|██████████| 12056/12056 [01:35<00:00, 126.02it/s]

Epoch 1: Train Loss = 0.3238


Epoch 1: Val Loss = 0.2979


Epoch 2/50: 100%|██████████| 12056/12056 [01:34<00:00, 127.20it/s]

Epoch 2: Train Loss = 0.2985


...

Epoch 49: Train Loss = 0.2805


Epoch 49: Val Loss = 0.2769


Epoch 50/50: 100%|██████████| 12056/12056 [01:35<00:00, 126.30it/s]

Epoch 50: Train Loss = 0.2806


Epoch 50: Val Loss = 0.2769
✅ Saved model to 'tcn_model_final_weather.pt'
✅ Saved predictions to 'bicikelj_test_predictions_tcn_weather.csv'


# SHAP

In [30]:
# --- SHAP analysis for trained TCN model ---

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import holidays
import shap
from tqdm import tqdm
from sklearn.metrics.pairwise import haversine_distances

# --- Constants ---
HISTORY_LEN = 48
PRED_HORIZON = 4
K_NEIGHBORS = 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
EMBED_DIM = 8
HIDDEN_DIM = 64
N_LAYERS = 3
DROPOUT = 0.2
BATCH_SIZE = 128
weather_features = ['temperature_2m', 'precipitation', 'windspeed_10m', 'cloudcover']

# --- Load training data and weather ---
df_train = pd.read_csv("bicikelj_train.csv")
meta = pd.read_csv("bicikelj_metadata.csv")
station_cols = df_train.columns[1:]

weather_df = pd.read_csv("weather_ljubljana.csv", skiprows=2)
weather_df = weather_df.rename(columns={
    'temperature_2m (°C)': 'temperature_2m',
    'precipitation (mm)': 'precipitation',
    'windspeed_10m (km/h)': 'windspeed_10m',
    'cloudcover (%)': 'cloudcover'
})
weather_df['time'] = pd.to_datetime(weather_df['time'])
df_train['timestamp'] = pd.to_datetime(df_train['timestamp']).dt.tz_localize(None)
df_train_merged = pd.merge(df_train, weather_df, left_on='timestamp', right_on='time', how='left')
df_train_merged[weather_features] = df_train_merged[weather_features].ffill().bfill()

# --- Station normalization ---
station_means = df_train_merged[station_cols].mean()
station_stds = df_train_merged[station_cols].std().replace(0, 1)
df_norm = df_train_merged.copy()
df_norm[station_cols] = (df_train_merged[station_cols] - station_means) / station_stds

# --- Weather normalization ---
weather_means = df_train_merged[weather_features].mean()
weather_stds = df_train_merged[weather_features].std().replace(0, 1)
df_norm[weather_features] = (df_train_merged[weather_features] - weather_means) / weather_stds

# --- Neighbors ---
coords = np.deg2rad(meta[['latitude', 'longitude']].values)
station_names = meta['name'].tolist()
dists = haversine_distances(coords, coords) * 6371
neighbors = {}
for i, name in enumerate(station_names):
    order = np.argsort(dists[i])
    nn_idx = [j for j in order if j != i][:K_NEIGHBORS]
    neighbors[name] = [station_names[j] for j in nn_idx]

# --- TCN model definition ---
class TemporalBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation, dropout):
        super().__init__()
        self.padding = (kernel_size - 1) * dilation
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size,
                               padding=self.padding, dilation=dilation)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.downsample = nn.Conv1d(in_channels, out_channels, 1) if in_channels != out_channels else None

    def forward(self, x):
        out = self.conv1(x)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        out = self.conv2(out)
        out = out[:, :, :-self.padding]
        out = self.relu(out)
        out = self.dropout(out)

        res = x if self.downsample is None else self.downsample(x)
        return out + res

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout, num_stations, embed_dim):
        super().__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_ch = input_size if i == 0 else num_channels[i - 1]
            out_ch = num_channels[i]
            layers += [TemporalBlock(in_ch, out_ch, kernel_size, dilation_size, dropout)]
        self.tcn = nn.Sequential(*layers)
        self.embedding = nn.Embedding(num_stations, embed_dim)
        self.head = nn.Sequential(
            nn.Linear(num_channels[-1] + embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_size)
        )

    def forward(self, x, station_id):
        x = x.permute(0, 2, 1)
        tcn_out = self.tcn(x)[:, :, -1]
        emb = self.embedding(station_id)
        combined = torch.cat([tcn_out, emb], dim=1)
        return self.head(combined)

# --- Load model ---
model = TCN(input_size=1 + K_NEIGHBORS + (8 + len(weather_features)),
            output_size=PRED_HORIZON,
            num_channels=[HIDDEN_DIM] * N_LAYERS,
            kernel_size=3,
            dropout=DROPOUT,
            num_stations=len(station_cols),
            embed_dim=EMBED_DIM).to(DEVICE)

model.load_state_dict(torch.load("tcn_model_final_weather.pt"))
model.eval()

# --- Prepare sample batch for SHAP ---
# We'll pick one station for simplicity (you can loop later)
s_name = station_cols[0]
station_to_idx = {name: i for i, name in enumerate(station_cols)}
s_idx = station_to_idx[s_name]
nn_idx = [station_to_idx[nn] for nn in neighbors[s_name]]

timestamps = pd.to_datetime(df_norm['timestamp'])
hour_sin = np.sin(2 * np.pi * timestamps.dt.hour / 24)
hour_cos = np.cos(2 * np.pi * timestamps.dt.hour / 24)
dow_sin = np.sin(2 * np.pi * timestamps.dt.dayofweek / 7)
dow_cos = np.cos(2 * np.pi * timestamps.dt.dayofweek / 7)
month_sin = np.sin(2 * np.pi * timestamps.dt.month / 12)
month_cos = np.sin(2 * np.pi * timestamps.dt.month / 12)
is_weekend = (timestamps.dt.dayofweek >= 5).astype(float)
slo_holidays = holidays.Slovenia()
is_holiday = timestamps.dt.date.astype(str).isin([str(d) for d in slo_holidays]).astype(float)

time_feats = np.stack([hour_sin, hour_cos, dow_sin, dow_cos,
                       month_sin, month_cos, is_weekend, is_holiday], axis=1)

bikes = df_norm[station_cols].values.astype(np.float32)
weather_array = df_norm[weather_features].values.astype(np.float32)

X_samples = []
Y_samples = []
SID_samples = []

N = len(df_norm)
for i in range(HISTORY_LEN, N - PRED_HORIZON + 1, 50):  # sample 1 every 50 → faster SHAP
    series = bikes[:, [s_idx] + nn_idx]
    full_feats = np.concatenate([series, time_feats, weather_array], axis=1)

    x_seq = full_feats[i - HISTORY_LEN:i]
    y_seq = bikes[i:i + PRED_HORIZON, s_idx]

    X_samples.append(x_seq)
    Y_samples.append(y_seq)
    SID_samples.append(s_idx)

X_samples = torch.tensor(np.stack(X_samples), dtype=torch.float32).to(DEVICE)
SID_samples = torch.tensor(SID_samples, dtype=torch.long).to(DEVICE)

# --- SHAP DeepExplainer ---
background = X_samples[:50]  # smaller background
background_sid = SID_samples[:50]

explainer = shap.DeepExplainer(model, [background, background_sid])
shap_values = explainer.shap_values([X_samples, SID_samples])

# --- Plot SHAP summary for horizon 0 ---
shap_array = shap_values[0]  # First horizon

# Collapse time axis: (batch, time, features) → (batch, time * features)
shap_array_collapsed = shap_array.reshape(shap_array.shape[0], -1)

# Feature names:
time_feat_names = ['hour_sin', 'hour_cos', 'dow_sin', 'dow_cos', 'month_sin', 'month_cos', 'is_weekend', 'is_holiday']
feature_names = (
    ['station_value'] +
    [f'neighbor_{i+1}' for i in range(K_NEIGHBORS)] +
    time_feat_names +
    weather_features
)

# For each timestep → repeat feature names
full_feature_names = []
for t in range(HISTORY_LEN):
    for fname in feature_names:
        full_feature_names.append(f"{fname}_t-{HISTORY_LEN - t}")

# --- SHAP summary plot ---
shap.summary_plot(shap_array_collapsed, feature_names=full_feature_names, show=True)


RuntimeError: Error(s) in loading state_dict for TCN:
	Unexpected key(s) in state_dict: "tcn.3.conv1.weight", "tcn.3.conv1.bias", "tcn.3.conv2.weight", "tcn.3.conv2.bias". 